In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_excel('./복합기 크롤링 중복제거.xlsx')

In [3]:
data.head()

,상품명,가격,스펙 목록,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,ApeosPort 2560,2125000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ApeosPort 3060,2200000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ApeosPort 3560,2528000,흑백 디지털 복합기,잉크젯,컬러 출력,복사,스캔,"인쇄 해상도: 4,800dpi",무한 패키지,연결방식: USB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ApeosPort C2060,2600000,컬러 디지털 복합기,인쇄,복사,스캔,컬러 속도: 23ppm,흑백 속도: 28ppm,인쇄 해상도: 600 x 600dpi,자동양면인쇄,...,가로: 699mm,세로: 742mm,높이: 759mm,무게: 121kg,NaN,NaN,NaN,NaN,NaN,NaN
4,ApeosPort C2560,2778170,컬러 디지털 복합기,인쇄,복사,스캔,팩스,컬러 속도: 23ppm,흑백 속도: 25ppm,인쇄 해상도: 1200 x 1200dpi,...,무게: 125kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.shape

(5901, 30)

In [5]:
data.fillna('')

,상품명,가격,스펙 목록,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,ApeosPort 2560,2125000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,,,,,,,,,,
1,ApeosPort 3060,2200000,흑백 디지털 복합기,잉크젯,포토프린터,컬러 출력,복사,스캔,팩스,"인쇄 해상도: 4,800dpi",...,,,,,,,,,,
2,ApeosPort 3560,2528000,흑백 디지털 복합기,잉크젯,컬러 출력,복사,스캔,"인쇄 해상도: 4,800dpi",무한 패키지,연결방식: USB,...,,,,,,,,,,
3,ApeosPort C2060,2600000,컬러 디지털 복합기,인쇄,복사,스캔,컬러 속도: 23ppm,흑백 속도: 28ppm,인쇄 해상도: 600 x 600dpi,자동양면인쇄,...,가로: 699mm,세로: 742mm,높이: 759mm,무게: 121kg,,,,,,
4,ApeosPort C2560,2778170,컬러 디지털 복합기,인쇄,복사,스캔,팩스,컬러 속도: 23ppm,흑백 속도: 25ppm,인쇄 해상도: 1200 x 1200dpi,...,무게: 125kg,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5896,코니카미놀타 bizhub C451,0,컬러 디지털 복합기,레이저,컬러 출력,복사,스캔,컬러 속도: 21ppm,흑백 속도: 21ppm,첫 장 인쇄: 10초,...,에어 프린팅,클라우드 프린팅,RAM: 1GB,터치스크린LCD,가로: 451mm,세로: 460mm,높이: 413mm,,,
5897,코니카미놀타 C022DN,0,컬러 디지털 복합기,잉크젯,컬러 출력,복사,스캔,컬러 속도(ISO): 6.8ipm,흑백 속도(ISO): 13ipm,인쇄 해상도: 4800 x 1200dpi,...,에어 프린팅,클라우드 프린팅,일반LCD,가로: 403mm,세로: 369mm,높이: 195mm,,,,
5898,코니카미놀타 C028DN,0,흑백 디지털 복합기,레이저젯,인쇄,컬러 속도: 38ppm,흑백 속도: 38ppm,인쇄 해상도: 3600dpi,자동양면인쇄,기본 급지용량: 650매,...,높이: 380mm,무게: 27.5kg,,,,,,,,
5899,코니카미놀타 C036DN,0,흑백 디지털 복합기,레이저,컬러 출력,복사,스캔,팩스,컬러 속도: 4ppm,흑백 속도: 18ppm,...,에코 프린팅,모바일 프린팅,NFC 탑재,RAM: 128MB,일반LCD,월 최대 인쇄량: 20000매,가로: 406mm,세로: 362mm,높이: 333.5mm,


In [6]:
###전처리 : 나눠진 데이터합치는 작업
cols = ['스펙 목록','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','Unnamed: 27','Unnamed: 28','Unnamed: 29']
data['keyword'] = data[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [7]:
###불필요한 행 삭제
data=data.drop(['스펙 목록','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','Unnamed: 27','Unnamed: 28','Unnamed: 29'], axis=1)
data.head()
data.to_excel('keyword.xlsx')

,상품명,가격,keyword
0,ApeosPort 2560,2125000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
1,ApeosPort 3060,2200000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
2,ApeosPort 3560,2528000,"흑백 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d..."
3,ApeosPort C2060,2600000,컬러 디지털 복합기 인쇄 복사 스캔 컬러 속도: 23ppm 흑백 속도: 2...
4,ApeosPort C2560,2778170,컬러 디지털 복합기 인쇄 복사 스캔 팩스 컬러 속도: 23ppm 흑백 속...


In [39]:
#tr-idf 단계 1 : 많이 쓰이는 단어이지만 내용이 없는 불필요한 단어 제거
tfidf = TfidfVectorizer(stop_words='english') #내용이 없는 불필요한 단어 제거
tfidf_matrix = tfidf.fit_transform(data['keyword']) #tr-idf 단계 2 : 핵심 키워드 추출
print(tfidf_matrix.shape) # 532개의 단어로 5091개의 제품을 표현하고 있음 

(5901, 532)


In [40]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [41]:
indices = pd.Series(data.index, index=data['상품명']).drop_duplicates()
print(indices.head())

상품명
ApeosPort 2560     0
ApeosPort 3060     1
ApeosPort 3560     2
ApeosPort C2060    3
ApeosPort C2560    4
dtype: int64


In [42]:
#tr-idf 단계 3 : 코싸인 유사도를 이용하여 단어 유사도를 구하기
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    # 모든 사무기기에 대해서 해당 사무기기와의 유사도 구하기.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 사무기기들을 정렬.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 사무기기를 받음.
    sim_scores = sim_scores[1:10]

    # 가장 유사한 10개의 사무기기의 인덱스를 받음.
    product_indices = [i[0] for i in sim_scores]
    


    # 가장 유사한 10개의 사무기기를 리턴.
    return data.iloc[product_indices]

In [43]:
get_recommendations('ApeosPort 2560')

,상품명,가격,keyword
130,Brother MFC-L6900DW,657720,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
585,Canon iR2206F,786000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
618,Canon IR2425,1096380,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
660,Canon iR2525,1649990,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
3614,Kyocera FS-1035MFP/DP,0,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
3655,Kyocera TASKalfa 4012i,2564000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
41,ApeosPort-V C2275,0,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
279,Canon IR ADV C3525(III),2806000,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
353,Canon IR ADV DX C3725,2593990,컬러 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...
